# arcpy demo - traffic simulation

## imports

In [5]:
from arcgis.features import GeoAccessor
import json
import logging
import os
import pandas as pd
import sqlite3 as sql
import warnings

In [6]:
def read_sqlite_as_sdf(db_filepath: str, select_statement: str, x_column: str='longitude', y_column: str='latitude'):
    """
    Reads the data from a sqlite database into main memory using a SQL statement.
    """
    with sql.connect(db_filepath) as connection:
        df = pd.read_sql_query(select_statement, connection)
        return GeoAccessor.from_xy(df, x_column, y_column)
    
def read_sqlite_to_featureclass(db_filepath: str, select_statement: str, x_column: str='longitude', y_column: str='latitude') -> GeoAccessor:
    """
    Reads the data from a sqlite database as an in memory feature class using a SQL statement.
    """
    filename_with_extension = os.path.basename(db_filepath)
    filename = os.path.splitext(filename_with_extension)[0]

    sdf = read_sqlite_as_sdf(db_filepath, select_statement, x_column, y_column)
    featureclass = sdf.spatial.to_featureclass("memory/" + filename)
    arcpy.management.ClearWorkspaceCache()
    return featureclass

In [7]:
# TODO: No hard coded path
feature_class = read_sqlite_to_featureclass(r"C:\Users\thkn.ESRI-DE\Downloads\traffic_data_1min_1610.sqlite", "SELECT * from agent_pos LIMIT 10;")
print(feature_class)

memory\traffic_data_1min_1610


## classes

In [8]:
class Trip(object):

    def __init__(self, name, trip_id, long, lat, trip_time, angle, distance, speed) -> None:
        self.name = name
        self.trip_id = trip_id
        self.long = long
        self.lat = lat
        self.trip_time = trip_time
        self.angle = angle
        self.distance = distance
        self.speed = speed

In [10]:
class MeasureTool(object):
     
    def create_geometry(self, longitude, latitude):

        point = arcpy.Point(longitude, latitude)
        point_geometry = arcpy.PointGeometry(point, arcpy.SpatialReference(4326))

        return point_geometry

    def measure(self, feature_class):
        
        trip_point_a = Trip("Point A", int, float, float, 0, float, float, float)

        feature_class_column_names = ["trip", "longitude", "latitude", "trip_time", "point_direction", "point_distance", "speed"]
        with arcpy.da.UpdateCursor(feature_class, feature_class_column_names) as cur:

            for row in cur:
                
                trip_point_b = Trip("Point B", row[0], row[1], row[2], self.round_datetime(row[3]), float, float ,float)
                if trip_point_b.trip_id == trip_point_a.trip_id:

                    trip_point_b = self.calculate_distance_speed(trip_point_b, trip_point_a)
                    row[4]=trip_point_b.angle
                    row[5]=trip_point_b.distance
                    row[6]=trip_point_b.speed

                    cur.updateRow(row)

                trip_point_a = Trip("Point A", trip_point_b.trip_id, trip_point_b.long, trip_point_b.lat,
                                    trip_point_b.trip_time, trip_point_b.angle, trip_point_b.distance, trip_point_b.speed)

        return feature_class

    def calculate_distance_speed(self, trip_point_b, trip_point_a):
        current_point = self.create_geometry(trip_point_b.lat, trip_point_b.long)
        last_point = self.create_geometry(trip_point_a.lat, trip_point_a.long)
        trip_point_b.angle, trip_point_b.distance = current_point.angleAndDistanceTo(last_point)
        
        differential_seconds = datetime.timedelta.total_seconds(trip_point_b.trip_time - trip_point_a.trip_time)
        trip_point_b.speed = trip_point_b.distance/differential_seconds * 3.6

        return trip_point_b

    def run(self, feature_class: str):
        # TODO: Implement the add fields for 'speed, point_distance, point_direction'
    
        sorted_feature_class = "memory/traffic_data"
        arcpy.Sort_management(feature_class, sorted_feature_class, [["trip", "ASCENDING"], ["trip_time", "ASCENDING"]])
        feature_class = arcpy.AddFields_management(in_table=sorted_feature_class, field_description=[["point_direction", "DOUBLE", "", "", "0", ""], ["point_distance", "DOUBLE", "", "", "0", ""], ["speed", "DOUBLE", "", "", "0", ""]])

        output = MeasureTool.measure(feature_class)
        return output

In [11]:
MeasureTool.run(feature_class)

TypeError: run() missing 1 required positional argument: 'feature_class'

# Analytics

In [ ]:
class SpaceTimeCube(object):
    
    def create_space_time_cube(feature_class: str, space_time_cube_path: str, time_interval: int, distance_interval: int):

        projected_feature_class = arcpy.Project_management(in_dataset = feature_class,
                                                           out_dataset = f"{feature_class}_projected_25832",
                                                           out_coor_system = arcpy.SpatialReference(25832))

        space_time_cube = arcpy.CreateSpaceTimeCube_stpm(projected_feature_class, 
                                       output_cube = f"{space_time_cube_path}/SpaceTimeTrafficCube.nc", 
                                       time_field ="trip_time",
                                       time_step_interval =  f"{time_interval} Minutes",
                                       distance_interval =  f"{distance_interval} Meters",
                                       aggregation_shape_type="HEXAGON_GRID")
        
        return space_time_cube

    def visualize_space_time_cube(space_time_cube_path:str, output_geodatabase:str):

        arcpy.VisualizeSpaceTimeCube3D_stpm(in_cube = space_time_cube_path,
                                            cube_variable = "COUNT",
                                            display_theme = "VALUE",
                                            output_features = output_geodatabase + "/SpaceTimeCubeVisualize")



In [ ]:
SpaceTimeCube.create_space_time_cube()

In [ ]:
class HotColdSpotsTool(object):

    def create_hot_cold_spots_space_time(space_time_cube_path: str, output_geodatabase:str, distance_interval: int):
    
        arcpy.stpm.EmergingHotSpotAnalysis(in_cube = space_time_cube_path,
                                           analysis_variable = "COUNT",
                                           output_features = output_geodatabase + "/SpaceTimeCube_EmergingHotSpotAnalysis",
                                           neighborhood_distance =  f"{distance_interval} Meters")

        arcpy.stpm.LocalOutlierAnalysis(in_cube = space_time_cube_path,
                                        analysis_variable = "COUNT",
                                        output_features = output_geodatabase + "/SpaceTimeCube_LocalOutlierAnalysis",
                                        neighborhood_distance =  f"{distance_interval} Meters")
        
    def create_hot_cold_spots_feature_class(feature_class: str, output_geodatabase:str, ):

        arcpy.FindPointClusters_gapro(input_points = feature_class,
                                      out_feature_class= f"{output_geodatabase}_PointCluster",
                                      clustering_method="DBSCAN",
                                      minimum_points= 100,
                                      search_distance="200 Meters",
                                      use_time="TIME",
                                      search_duration="10 Minutes")

        arcpy.CalculateDensity_gapro(input_layer = feature_class,
                                     out_feature_class = f"{output_geodatabase}_CalculateDensity",
                                     bin_type = "HEXAGON",
                                     bin_size = "200 Meters",
                                     weight = "UNIFORM",
                                     neighborhood_size = "300 Meters",
                                     area_unit_scale_factor = "SQUARE_METERS",
                                     time_step_interval = "10 Minutes")
        
        arcpy.FindHotSpots_gapro(point_layer = feature_class,
                                 out_feature_class = f"{output_geodatabase}_FindHotSpots",
                                 bin_size = "200 Meters",
                                 neighborhood_size = "300 Meters",
                                 time_step_interval = "10 Minutes")

HotColdSpotsTool.create_hot_cold_spots_space_time()